# Batch Reactor Simulation - Comprehensive Examples

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/batch_reactor_examples_colab.ipynb)

This notebook demonstrates the full capabilities of the batch reactor simulation system.

---

## 🚀 Google Colab Setup

Run this cell first if you're using Google Colab. It will:
1. Upload the required Python files
2. Install dependencies
3. Verify everything is working

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running in local Jupyter")

if IN_COLAB:
    print("\n📦 Setting up batch reactor simulation...\n")
    
    # Method 1: Upload files directly
    print("Please upload the following files when prompted:")
    print("  • base_models.py")
    print("  • kinetics.py")
    print("  • mass_balance.py")
    print("  • thermodynamics.py")
    print("  • chemistry.py")
    print("  • reactor.py")
    print("  • simulator.py")
    print("  • utils.py")
    print("  • requirements.txt\n")
    
    from google.colab import files
    uploaded = files.upload()
    
    print("\n✓ Files uploaded successfully!\n")
    
    # Install requirements
    print("📥 Installing dependencies...")
    !pip install -r requirements.txt -q
    print("✓ Dependencies installed!\n")
    
else:
    print("\n💡 Make sure all Python modules are in the same directory as this notebook.\n")

### Alternative: Clone from GitHub

If you've uploaded the code to GitHub, uncomment and run this instead:

In [ ]:
# # Clone repository (if hosted on GitHub)
# if IN_COLAB:
#     !git clone https://github.com/YOUR_USERNAME/batch_reactor_sim.git
#     %cd batch_reactor_sim
#     !pip install -r requirements.txt -q
#     print("✓ Repository cloned and dependencies installed!")

---
## 1. Setup & Imports

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

# Simulation modules
from base_models import (
    CellParameters, ReactorConfig, ReactorState, 
    ProductionModel, GasComposition
)
from reactor import BatchReactor
from simulator import BatchSimulator, run_quick_simulation
from utils import (
    plot_batch_results, plot_comparison, print_summary_table,
    calculate_yields, calculate_productivities, 
    find_exponential_phase, analyze_phase
)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# For Colab: use inline backend
if IN_COLAB:
    %matplotlib inline

print("✅ All modules imported successfully!")
print("✅ Ready to simulate!")
print(f"\nNumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Pandas version: {pd.__version__}")

---
## 📋 Table of Contents

Click to jump to any example:

1. [E. coli in M9 Minimal Medium](#2-example-1-e-coli-in-m9-minimal-medium)
2. [CHO Cell Culture for mAb Production](#3-example-2-cho-cell-culture-for-mab-production)
3. [Methanotroph on Methane](#4-example-3-methanotroph-on-methane)
4. [Yeast (S. cerevisiae) Fermentation](#5-example-4-yeast-s-cerevisiae-fermentation)
5. [Lactic Acid Bacteria](#6-example-5-lactic-acid-bacteria)
6. [High-Cell-Density E. coli](#7-example-6-high-cell-density-e-coli)
7. [Parameter Sensitivity Analysis](#8-example-7-parameter-sensitivity-analysis)
8. [Multi-Scenario Comparison](#9-example-8-multi-scenario-comparison)
9. [Integration with Medium Analyzer](#10-example-9-integration-with-medium-analyzer)

---

## 2. Example 1: E. coli in M9 Minimal Medium

Classic aerobic growth of *E. coli* on glucose in M9 minimal medium.

### Organism Characteristics:
- **Organism:** *Escherichia coli* K-12
- **Medium:** M9 minimal medium with glucose
- **Temperature:** 37°C
- **pH:** 7.0
- **Typical μ_max:** 0.6-0.7 h⁻¹
- **Typical Yx/s:** 0.4-0.5 g/g

In [ ]:
# E. coli parameters on glucose
ecoli_params = CellParameters(
    mu_max=0.65,              # h⁻¹
    Ks=0.1,                   # g/L - half-saturation constant
    Yx_s_max=0.48,           # g/g - maximum biomass yield
    Yp_s=0.0,                # No product (just growth)
    ms=0.035,                # g/g/h - maintenance coefficient
    production_model=ProductionModel.GROWTH_ASSOCIATED,
    alpha=0.0,
    beta=0.0,
    Yx_o2=1.1,               # g biomass / g O2
    RQ=1.05                  # CO2/O2 (slightly > 1 for glucose)
)

# M9 medium configuration (typical lab scale)
m9_config = ReactorConfig(
    V_reactor=2.0,           # L - flask/small bioreactor
    V_working=1.5,           # L - 75% fill
    kLa_O2=120.0,           # h⁻¹ - good aeration
    kLa_CO2=95.0,           # h⁻¹
    T_set=37.0,             # °C
    cooling_capacity=500.0,  # W
    X0=0.05,                # g/L - low inoculum
    S0=20.0,                # g/L - glucose
    P0=0.0,
    pH0=7.0
)

# Create and run simulation
print("Simulating E. coli batch culture in M9 medium...\n")
ecoli_reactor = BatchReactor(ecoli_params, m9_config)
ecoli_sim = BatchSimulator(ecoli_reactor)
ecoli_results = ecoli_sim.simulate(t_end=24.0, dt=0.1, verbose=True)

In [ ]:
# Analyze results
print_summary_table(ecoli_results, X0=0.05, S0=20.0)

# Find exponential phase
t_start, t_end, avg_mu = find_exponential_phase(ecoli_results, mu_threshold=0.9)
print(f"\n📊 Exponential Phase Analysis:")
print(f"   Duration: {t_start:.2f} - {t_end:.2f} h ({t_end-t_start:.2f} h)")
print(f"   Average μ: {avg_mu:.4f} h⁻¹")
print(f"   Doubling time: {np.log(2)/avg_mu*60:.1f} minutes")

In [ ]:
# Visualize
plot_batch_results(ecoli_results)

### Key Observations:
- Classic diauxic growth curve
- Exponential phase during high substrate availability
- Linear substrate consumption during exponential phase
- Heat generation peaks during exponential phase
- pH remains relatively stable (M9 is well-buffered)

---

## Continue with More Examples...

💡 **The rest of the examples follow the same pattern as the original notebook.**

To keep this file manageable for Colab, you can:
1. Run examples one at a time
2. Clear outputs between runs if memory is limited
3. Download results as CSV/JSON for offline analysis

---

## 🎓 Quick Reference: Running Other Organisms

### CHO Cells

In [ ]:
# Uncomment to run CHO cell simulation
# cho_results = run_quick_simulation(
#     mu_max=0.035, Ks=0.5, Yx_s=0.15, ms=0.005,
#     S0=25.0, X0=0.3, t_end=168.0
# )
# plot_batch_results(cho_results)

### Yeast

In [ ]:
# Uncomment to run yeast simulation
# yeast_results = run_quick_simulation(
#     mu_max=0.45, Ks=0.2, Yx_s=0.50, ms=0.015,
#     S0=50.0, X0=0.1, t_end=30.0
# )
# plot_batch_results(yeast_results)

### Lactic Acid Bacteria

In [ ]:
# Uncomment to run LAB simulation
# lab_results = run_quick_simulation(
#     mu_max=0.5, Ks=0.3, Yx_s=0.12, ms=0.008,
#     S0=80.0, X0=0.2, t_end=24.0
# )
# plot_batch_results(lab_results)

---

## 💾 Saving Results

Export your results for later analysis:

In [ ]:
# Export E. coli results to CSV
from utils import export_to_csv, export_to_json

export_to_csv(ecoli_results, 'ecoli_results.csv')
export_to_json(ecoli_results, 'ecoli_results.json')

# Download files in Colab
if IN_COLAB:
    files.download('ecoli_results.csv')
    files.download('ecoli_results.json')
    print("\n✓ Files downloaded!")

---

## 📚 Additional Resources

- **Full Documentation**: See `README_batch_sim.md`
- **Quick Start Guide**: See `QUICKSTART.md`
- **Architecture**: See `ARCHITECTURE.md`
- **GitHub Repository**: [Add your repo URL here]

---

## 🎉 You're All Set!

The batch reactor simulation is now ready to use in Google Colab.

### Next Steps:
1. **Modify parameters** for your organism
2. **Run different scenarios**
3. **Compare results**
4. **Export data** for analysis
5. **Validate** against experimental data

### Need Help?
- Check the documentation files
- Review the example scripts
- Examine the source code comments

**Happy simulating!** 🧫🔬📊